<a href="https://colab.research.google.com/github/soutrik71/pytorch_classics/blob/main/APTorch7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The focus of this notebook is to implement LSTM for sequence generation tasks
1. Text Generation using LSTM Networks (Character-based RNN)
2. Text Generation using PyTorch LSTM Networks (Character Embeddings)
3. Sequence generation at word level using LSTM Networks (Word Embeddings)
3. Application of pre-trained embedding models for the better representation of words

In [1]:
# !pip install portalocker
# !pip install torchview
# !pip install torcheval
# !pip install scikit-plot
# !pip install lime

In [2]:
import os
os.chdir(
    r"/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run/code/Users/soutrik.chowdhury/pytorch_classics"
)

In [3]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torch.utils.data import DataLoader, TensorDataset,Dataset
from torchtext import data
from torchtext import datasets
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import re
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset
from torchsummary import summary
from torchview import draw_graph
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from torcheval.metrics import MulticlassAccuracy,BinaryAccuracy
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix
import scikitplot as skplt

In [5]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [6]:
# Set manual seed since nn.Parameter are randomly initialzied
set_seed(42)
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
batch_size = 1024
epochs = 10
lr = 1e-4
embedding = False

Random seed set as 42
cuda


## Basic DataPrep

In this section, we undertake data preparation for training the neural network using a character-based approach. Specifically, we adopt a fixed sequence length of 100 characters, where the network's task is to predict the subsequent character given this sequence. Employing an embeddings technique, we encode text data by assigning a unique real-valued vector to each character.

The data preparation procedure is as follows:

* Loading Text Examples and Creating Vocabulary: Iterate through all text examples to construct a vocabulary, mapping each character to a distinct integer index. This vocabulary
facilitates character representation in a numerical format.

* Organizing Data with a Sliding Window: Implement a sliding window mechanism to organize the data. For every text example, we slide a window of 100 characters. The first 100 characters serve as input features (X), while the 101st character becomes the target value (Y). This process continues by shifting the window one character at a time until the end of the text example.

* Conversion to Integer Indices: Retrieve integer indices corresponding to characters in both data features and target values based on the previously constructed vocabulary. This step transforms characters into their corresponding numerical representations.

* Embeddings Assignment: Each unique integer index, representing a specific character in the data features, is associated with a real-valued vector known as an embedding. These embeddings provide a continuous representation of characters, facilitating numerical computation within the neural network. This is optional

### Data loading

In [7]:
train_dataset, valid_dataset, test_dataset = datasets.PennTreebank()

In [8]:
next(iter(train_dataset.shuffle()))

'instead new york city police seized the stolen goods and mr. <unk> avoided jail'

In [9]:
def info(x):
  return len(x)

elem_ls = list(train_dataset.map(info))

/home/azureuser/.local/lib/python3.10/site-packages/torch/utils/data/datapipes/iter/combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [10]:
print(len(elem_ls)) # total 42k elements
print(max(elem_ls)) # max length of each element
print(min(elem_ls)) # min length of each element

42068
518
2


We construct a vocabulary of unique characters using build_vocab_from_iterator() from torchtext's 'vocab' sub-module. Our custom function build_vocabulary() serves as an iterator, looping through datasets and examples to yield character lists. Special handling ensures the '<unk>' token, representing unknown characters, is counted as a single token rather than individual characters.






In [11]:
def build_vocabulary(datasets):
  for dataset in datasets:
    for text in dataset:

      if "unk" in text:
        texts = text.split("<unk>")
        total = list(texts[0].lower())
        for t in texts[1:]:
            total.extend(["<unk>", ] + list(t.lower()))
        yield total

      else:
        yield list(text.lower())

In [12]:
vocab = build_vocab_from_iterator(build_vocabulary([train_dataset, valid_dataset, test_dataset]), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [13]:
len(vocab)

47

In [14]:
print(vocab.get_itos()) # character level tokenization

['<unk>', ' ', 'e', 't', 'a', 'n', 'o', 'i', 's', 'r', 'h', 'l', 'd', 'c', 'u', 'm', 'f', 'p', 'g', 'y', 'b', 'w', 'v', 'k', '.', "'", 'x', 'j', '$', '-', 'q', 'z', '&', '0', '1', '9', '3', '#', '2', '8', '5', '\\', '7', '6', '/', '4', '*']


In [15]:
print(vocab.get_stoi()) # dictionary mapping token to indices

{'4': 45, '/': 44, '7': 42, '8': 39, '2': 38, '#': 37, '9': 35, '1': 34, 'z': 31, 'q': 30, '-': 29, '6': 43, '3': 36, 'r': 9, 's': 8, 'd': 12, 'k': 23, 'n': 5, 'h': 10, '*': 46, 'u': 14, '0': 33, 'p': 17, 't': 3, 'i': 7, '\\': 41, '5': 40, 'a': 4, 'e': 2, 'j': 27, '&': 32, 'v': 22, 'o': 6, '<unk>': 0, '.': 24, 'c': 13, 'm': 15, 'f': 16, 'l': 11, 'g': 18, 'y': 19, 'b': 20, 'w': 21, ' ': 1, 'x': 26, "'": 25, '$': 28}


Preparing the sequential data for training with sliding window approach and window size of 10 characters

In [16]:
seq_len = 35
train_records_max = 5000
X_train_full, y_train_full = [], []
X_val_full , y_val_full = [], []

In [ ]:
# train data prep
for idex, text in enumerate(train_dataset):
  print(text)
  print("\n")
  for i in range(len(text) - seq_len):
    inp_rec = list(text[i:i+seq_len].lower())
    op_rec = text[i+seq_len].lower()

    if len(op_rec) == 0:
      break

    X_train_full.append(vocab(inp_rec))
    y_train_full.append(vocab[op_rec])

  if idex > train_records_max:
    break

In [18]:
print(len(X_train_full))
print(len(y_train_full))

423585
423585


In [ ]:
# validation dataset prep
for idex, text in enumerate(valid_dataset):
  print(text)
  print("\n")
  for i in range(len(text) - seq_len):
    inp_rec = list(text[i:i+seq_len].lower())
    op_rec = text[i+seq_len].lower()

    if len(op_rec) == 0:
      break

    X_val_full.append(vocab(inp_rec))
    y_val_full.append(vocab[op_rec])

In [20]:
print(len(X_val_full))
print(len(y_val_full))

273982
273982


In [21]:
X_train = torch.tensor(X_train_full, dtype=torch.float32)
y_train = torch.tensor(y_train_full)
print(f"The shape of X_train is {X_train.shape}") # n records with k elements in each
print(f"The shape of Y_train is {y_train.shape}") # n records with 1 element in each

The shape of X_train is torch.Size([423585, 35])
The shape of Y_train is torch.Size([423585])


In [22]:
X_val = torch.tensor(X_val_full, dtype=torch.float32)
y_val = torch.tensor(y_val_full)
print(f"The shape of X_train is {X_val.shape}") # n records with k elements in each
print(f"The shape of Y_train is {y_val.shape}") # n records with 1 element in each

The shape of X_train is torch.Size([273982, 35])
The shape of Y_train is torch.Size([273982])


In [23]:
if not embedding:
  X_train = X_train.unsqueeze(dim=-1)
  X_val = X_val.unsqueeze(dim=-1)

Dataloader part

In [24]:
vectorized_train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(vectorized_train_dataset, batch_size=1024, shuffle=False)

vectorized_valid_dataset = TensorDataset(X_val, y_val)
valid_loader = DataLoader(vectorized_valid_dataset, batch_size=1024, shuffle=False)

In [25]:
for x, y in train_loader:
  print(x.shape)
  print(y.shape)
  break

torch.Size([1024, 35, 1])
torch.Size([1024])


## Modelling Building using Character based RNN

The network includes 2 LSTM layers with an output size of 256 each, followed by a linear layer. Stacking these LSTM layers enhances sequence learning. The output of the second LSTM layer feeds into the linear layer, whose output units match the vocabulary size

In [26]:
hidden_dim = 256
n_layers=2

class LSTMTextGenerator(nn.Module):
    def __init__(self):
        super(LSTMTextGenerator, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, len(vocab))

    def forward(self, X_batch):
      # init weights
      hidden = torch.randn(n_layers, len(X_batch), hidden_dim).to(device)
      carry = torch.randn(n_layers, len(X_batch), hidden_dim).to(device)

      output, (hidden, carry) = self.lstm(X_batch, (hidden, carry))
      return self.linear(output[:,-1])

In [27]:
text_generator_lstm = LSTMTextGenerator().to(device)

In [28]:
for layer in text_generator_lstm.children():
    print("Layer : {}".format(layer))
    print("Parameters : ")
    for param in layer.parameters():
        print(param.shape)
    print("\n")

Layer : LSTM(1, 256, num_layers=2, batch_first=True)
Parameters : 
torch.Size([1024, 1])
torch.Size([1024, 256])
torch.Size([1024])
torch.Size([1024])
torch.Size([1024, 256])
torch.Size([1024, 256])
torch.Size([1024])
torch.Size([1024])


Layer : Linear(in_features=256, out_features=47, bias=True)
Parameters : 
torch.Size([47, 256])
torch.Size([47])




In [29]:
def train_module(
    model: torch.nn.Module,
    device: torch.device,
    train_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    train_losses: list,
):

    # setting model to train mode
    model.train()
    pbar = tqdm(train_dataloader)

    # batch metrics
    train_loss = 0
    processed_batch = 0

    for idx, (data, label) in enumerate(pbar):
        # setting up device
        data = data.to(device)
        label = label.to(device)

        # forward pass output
        preds = model(data)

        # calc loss
        loss = criterion(preds, label)
        train_loss += loss.item()
        # print(f"training loss for batch {idx} is {loss}")

        # backpropagation
        optimizer.zero_grad()  # flush out  existing grads
        loss.backward()  # back prop of weights wrt loss
        optimizer.step()  # optimizer step -> minima

        # updating batch count
        processed_batch += 1

        pbar.set_description(f"Avg Train Loss: {train_loss/processed_batch}")

    # updating epoch metrics
    train_losses.append(train_loss / processed_batch)

    return train_losses

In [30]:
def test_module(
    model: torch.nn.Module,
    device: torch.device,
    test_dataloader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    test_losses,
):
    # setting model to eval mode
    model.eval()
    pbar = tqdm(test_dataloader)

    # batch metrics
    test_loss = 0
    processed_batch = 0

    with torch.inference_mode():
        for idx, (data, label) in enumerate(pbar):
            data, label = data.to(device), label.to(device)
            # predictions
            preds = model(data)
            # print(preds.shape)
            # print(label.shape)

            # loss calc
            loss = criterion(preds, label)
            test_loss += loss.item()

            # updating batch count
            processed_batch += 1

            pbar.set_description(f"Avg Test Loss: {test_loss/processed_batch}")

        # updating epoch metrics
        test_losses.append(test_loss / processed_batch)

    return test_losses

In [31]:
optimizer = optim.Adam(text_generator_lstm.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [32]:
%%time

# Place holders----
train_losses = []
test_losses = []

for epoch in range(0,epochs):
  print(f'Epoch {epoch}')
  train_losses = train_module(text_generator_lstm, device, train_loader, optimizer, criterion, None, train_losses, None)
  test_losses = test_module(text_generator_lstm, device, valid_loader, criterion, None, test_losses, None)

Epoch 0


  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0/414 [00:00<?, ?it/s]

Avg Test Loss: 2.8952118158340454: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [00:07<00:00, 38.16it/s]


Epoch 1


Avg Test Loss: 2.820933357103547: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [00:07<00:00, 38.14it/s]


...

Epoch 8


Avg Test Loss: 2.3514605075565735: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [00:07<00:00, 37.64it/s]


Epoch 9


Avg Test Loss: 2.3250607901544713: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [00:06<00:00, 38.54it/s]

CPU times: user 4min 21s, sys: 3.05 s, total: 4min 24s
Wall time: 4min 23s


## Evaluation
The logic starts with the initial randomly selected sequence and makes the next character prediction. It then removes the first character from the sequence and adds a newly predicted character at the end. Then, it makes another prediction and the process repeats for 100 characters.

In [33]:
idx = random.randint(0, len(X_train))
pattern = X_train[idx].numpy().astype(int).flatten().tolist() # list of tokens matched with torch text
print("Initial Pattern : {}".format("".join(vocab.lookup_tokens(pattern))))

Initial Pattern : iami fla. company that packages and


In [34]:
generated_text = []
# genererate 100 characters and also offsetting one character after every iterations to maintain the seq length
for i in range(100):
  batch = torch.tensor(pattern, dtype=torch.float32).reshape(1, seq_len, 1).to(device)
  model_op = text_generator_lstm(batch)
  # print(model_op.shape) # 47 is the vocab size
  predicted_index = model_op.argmax(dim=-1).squeeze().cpu().item()
  generated_text.append(predicted_index) ## Add token index to result
  pattern.append(predicted_index) ## Add token index to original pattern
  pattern = pattern[1:] ## Resize pattern to bring again to seq_length l


In [35]:
print("Generated Text : {}".format("".join(vocab.lookup_tokens(generated_text))))

Generated Text :  the the the the the the the the the the the the the the the the the the the the the the the the the


The model is producing some random text but english text but in repeations
and even after mutiple training yet it produces similar random text

## Model Building using self trained embeddings

We have used character-based approach for our case which means that our network takes a list of characters as input and returns the next character that it thinks should come next. We can also design models that take a list of words as input and predicts the next word. For encoding text data, we have used character embeddings approach which assigns a real-valued vector to each token (character)

Network Architecture:

* Embedding Layer: 100 embedding length, input (batch_size, seq_length), output (batch_size, seq_length, 100).
* LSTM Layer 1 & 2: 256 hidden dimensions, input (batch_size, seq_length, embed_len), output (batch_size, seq_length, 256).
* Linear Layer: Output units match vocabulary length, input (batch_size, seq_length, 256), output (batch_size, vocab_len).
* Embedding Layer:

  Utilizes Embedding() constructor with vocab length and 100 embedding length.\
  Transforms input shape to (batch_size, seq_length, embed_len).

* LSTM Layers:

  LSTM Layer 1 processes embedding output with 256 hidden dimensions.\
  LSTM Layer 2 processes LSTM 1 output with 256 hidden dimensions.

* Linear Layer:\
  Transforms LSTM 2 output to (batch_size, vocab_len), representing predictions.

* Initialization & Verification:\
  Initialized network and examined weights/biases.\
  Conducted forward pass with sample data for validation.

In [36]:
X_train = torch.tensor(X_train_full, dtype=torch.int64)
y_train = torch.tensor(y_train_full)
print(f"The shape of X_train is {X_train.shape}") # n records with k elements in each
print(f"The shape of Y_train is {y_train.shape}") # n records with 1 element in each

The shape of X_train is torch.Size([423585, 35])
The shape of Y_train is torch.Size([423585])


In [37]:
X_val = torch.tensor(X_val_full, dtype=torch.int64)
y_val = torch.tensor(y_val_full)
print(f"The shape of X_train is {X_val.shape}") # n records with k elements in each
print(f"The shape of Y_train is {y_val.shape}") # n records with 1 element in each

The shape of X_train is torch.Size([273982, 35])
The shape of Y_train is torch.Size([273982])


In [38]:
# new data loader
vectorized_train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(vectorized_train_dataset, batch_size=1024, shuffle=False)

vectorized_valid_dataset = TensorDataset(X_val, y_val)
valid_loader = DataLoader(vectorized_valid_dataset, batch_size=1024, shuffle=False)

In [39]:
embed_len = 100
hidden_dim = 256
n_layers=2

class LSTMTextGenerator_Embed(nn.Module):
    def __init__(self):
        super(LSTMTextGenerator_Embed, self).__init__()
        self.word_embedding = nn.Embedding(num_embeddings= 47, embedding_dim=embed_len)
        self.lstm = nn.LSTM(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, len(vocab))

    def forward(self, X_batch):
        embeddings = self.word_embedding(X_batch)

        hidden, carry = torch.randn(n_layers, len(X_batch), hidden_dim).to(device), torch.randn(n_layers, len(X_batch), hidden_dim).to(device)
        output, (hidden, carry) = self.lstm(embeddings, (hidden, carry))
        return self.linear(output[:,-1])

In [40]:
text_generator_lstm_embd = LSTMTextGenerator_Embed().to(device)

In [41]:
optimizer = optim.Adam(text_generator_lstm_embd.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [42]:
%%time

# Place holders----
train_losses = []
test_losses = []

for epoch in range(0,epochs):
  print(f'Epoch {epoch}')
  train_losses = train_module(text_generator_lstm_embd, device, train_loader, optimizer, criterion, None, train_losses, None)
  test_losses = test_module(text_generator_lstm_embd, device, valid_loader, criterion, None, test_losses, None)

Epoch 0


Avg Train Loss: 3.8494882583618164:   0%|██▏                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 1/414 [00:00<05:28,  1.26it/s]

Avg Test Loss: 2.4405801331818995: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [00:07<00:00, 36.83it/s]


Epoch 1


Avg Test Loss: 2.157608670974845: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [00:07<00:00, 38.03it/s]


...

Epoch 8


Avg Test Loss: 1.6765732146910768: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [00:07<00:00, 37.60it/s]


Epoch 9


Avg Test Loss: 1.6440599760902461: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [00:07<00:00, 37.79it/s]

CPU times: user 4min 35s, sys: 2.57 s, total: 4min 38s
Wall time: 4min 36s


In [43]:
idx = random.randint(0, len(X_train))
pattern = X_train[idx].numpy().astype(int).flatten().tolist() # list of tokens matched with torch text
print("Initial Pattern : {}".format("".join(vocab.lookup_tokens(pattern))))

Initial Pattern : rs. yeargin did although most are n


In [44]:
generated_text = []
# genererate 100 characters and also offsetting one character after every iterations to maintain the seq length
for i in range(100):
  batch = torch.tensor(pattern, dtype=torch.int64).reshape(1, seq_len).to(device)
  model_op = text_generator_lstm_embd(batch)
  # print(model_op.shape) # 47 is the vocab size
  predicted_index = model_op.argmax(dim=-1).squeeze().cpu().item()
  generated_text.append(predicted_index) ## Add token index to result
  pattern.append(predicted_index) ## Add token index to original pattern
  pattern = pattern[1:] ## Resize pattern to bring again to seq_length l


In [45]:
print("Generated Text : {}".format("".join(vocab.lookup_tokens(generated_text))))

Generated Text :  n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n n


References:

https://coderzcolumn.com/tutorials/artificial-intelligence/text-generation-using-pytorch-lstm-networks-and-character-embeddings\
https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-text-generation-using-lstm-networks

## Text Generation Word level

Shortcomings of Character-Level Text Generation:

Lack of Context: Operates at a granular level, focusing on individual characters, which hinders the ability to maintain broader context necessary for coherent text generation.
Prone to Random Output: In longer sequences, more susceptible to generating gibberish or nonsensical text due to the absence of word-level semantics.

Promise of Word Level Text Generation:

Improved Contextual Understanding: Operates at a higher linguistic level, capturing relationships between words, phrases, and sentences for more coherent text generation, suitable for tasks requiring longer sequences.
Enhanced Readability: Produces text that is more readable and human-like by adhering to linguistic conventions, making it suitable for applications like story generation and content creation.

In [7]:
with open('data/alice/alice.txt', 'r', encoding='utf-8') as file:
  train_text = file.read()


The process would primarily involve:
- First, we will prepare the dataset. This includes the preparation of vocabulary, a dictionary for mapping words to integers, and a reverse dictionary as well.
- Second, we need to prepare the data loaders where the input will be sentences to a certain length and the targets will be the same sentences but shifted one place to the right.
- Third, we need to prepare the LSTM model.
- Finally, we will carry out the training and inference using the trained model.

In [8]:
len(train_text)

145190

#### Preprocessing of text io rudimentary way

In [9]:
from collections import Counter

In [10]:
words = train_text.split()
words_counter = Counter(words)

In [11]:
print(len(words))
print(words_counter.most_common(10))

26563
[('the', 1510), ('and', 715), ('to', 703), ('a', 606), ('of', 493), ('she', 484), ('said', 416), ('it', 348), ('in', 347), ('was', 328)]


In [12]:
# unique word count
vocab_ls = words_counter.keys()
vocab_size = len(vocab_ls)

In [13]:
def words_to_index():
    """Convert words to index"""
    op = {word: i for i, word in enumerate(vocab_ls)}

    return op


def index_to_word():
    """Convert index to words"""
    op = words_to_index()
    return {i: word for word, i in op.items()}

In [14]:
words_to_idx = words_to_index()
idx_to_word = index_to_word()

In [15]:
idx_to_word[0]

'\ufeffProject'

In [16]:
# we are creating each sample of 64 words with first 63 is training and last one is always testing
# next set of words starts based on the step size
SEQUENCE_LENGTH = 64
samples = [words[i:i+SEQUENCE_LENGTH+1] for i in range(0,len(words)-SEQUENCE_LENGTH,1)]

In [17]:
print(len(samples))

26499


In [18]:
samples[0][-2:]

['June', '25,']

In [19]:
samples[1][-1:]

['2008']

* Next for every sample we will pass it through a custom datset generator which will take last but 1 element as train data and the last elenment as target
* The class accepts the samples which is a list of lists containing sequences of 64 words each. Along with that, it also accepts the word_to_int dictionary for mapping.

In [47]:
batch_size = 64
epochs = 50
lr = 1e-4

In [48]:
class AliceDataset(Dataset):
    def __init__(self, samples, words_to_idx):
        self.samples = samples
        self.words_to_idx = words_to_idx

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        X = torch.tensor(
            [self.words_to_idx[word] for word in sample[:-1]], dtype=torch.int64
        )
        y = torch.tensor(
            [self.words_to_idx[word] for word in sample[1:]], dtype=torch.int64
        )
        return X, y

In [49]:
alice_dataset = AliceDataset(samples, words_to_idx)
for data , target in alice_dataset:
    print(data.shape)
    print(target.shape)
    break

torch.Size([64])
torch.Size([64])


In the __getitem__ method, we extract one sample based on the index. input_seq is a sequence of integers that corresponds to the words of the sample excluding the last word. This will be used as the input to the LSTM. target_seq is a sequence of integers that corresponds to the words of the sample excluding the first word. This is the target sequence that the model will try to predict. Each element in the target_seq is the next word following the corresponding element in the input_seq.

In [50]:
# next we create the batchloader
train_loader = torch.utils.data.DataLoader(
    alice_dataset, shuffle=True, batch_size=batch_size
)

In [51]:
for data, target in train_loader:
    print(data.shape)
    print(target.shape)
    break

torch.Size([64, 64])
torch.Size([64, 64])


In [52]:
class TextGenerationLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers):
        super(TextGenerationLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
        )
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def forward(self, x, hidden=None):
        if hidden == None:
            hidden = self.init_hidden(x.shape[0])
        x = self.embedding(x)
        out, (h_n, c_n) = self.lstm(x, hidden)
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.fc(out)
        return out, (h_n, c_n)

    def init_hidden(self, batch_size):
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return h0, c0

In [54]:
embedding_dim = 32
hidden_size = 32
num_layers = 2

In [55]:
lstm_word_model = TextGenerationLSTM(vocab_size, embedding_dim, hidden_size, num_layers).to(device)
print(lstm_word_model)

TextGenerationLSTM(
  (embedding): Embedding(5335, 32)
  (lstm): LSTM(32, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=5335, bias=True)
)


In [56]:
def train_module(
    model: torch.nn.Module,
    device: torch.device,
    train_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    train_losses: list,
):

    # setting model to train mode
    model.train()
    pbar = tqdm(train_dataloader)

    # batch metrics
    train_loss = 0
    processed_batch = 0

    for idx, (data, label) in enumerate(pbar):
        # setting up device
        data = data.to(device)
        label = label.to(device).view(-1)
        # print(label.shape)

        # forward pass output
        preds, _ = model(data)
        # print(preds.shape)

        # calc loss
        loss = criterion(preds, label)
        train_loss += loss.item()
        # print(f"training loss for batch {idx} is {loss}")

        # backpropagation
        optimizer.zero_grad()  # flush out  existing grads
        loss.backward()  # back prop of weights wrt loss
        optimizer.step()  # optimizer step -> minima

        # updating batch count
        processed_batch += 1

        pbar.set_description(f"Avg Train Loss: {train_loss/processed_batch}")

    # updating epoch metrics
    train_losses.append(train_loss / processed_batch)

    return train_losses

In [57]:
optimizer = optim.Adam(lstm_word_model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

TRAINED FOR 200 epochs

In [63]:
%%time

# Place holders----
train_losses = []
test_losses = []

for epoch in range(0,epochs):
  print(f'Epoch {epoch}')
  train_losses = train_module(lstm_word_model, device, train_loader, optimizer, criterion, train_losses)

Epoch 0


Avg Train Loss: 3.2510738126162826:   5%|████████████████████████████████████████████▌                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 20/415 [00:00<00:04, 90.52it/s]

Avg Train Loss: 3.2535275453544523: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 415/415 [00:04<00:00, 98.25it/s]


Epoch 1


Avg Train Loss: 3.2403508898723556: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 415/415 [00:04<00:00, 97.27it/s]


...

Epoch 48


Avg Train Loss: 2.758033669713032: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 415/415 [00:04<00:00, 98.96it/s]


Epoch 49


Avg Train Loss: 2.748944199803364: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 415/415 [00:04<00:00, 100.25it/s]

CPU times: user 3min 22s, sys: 19.6 s, total: 3min 42s
Wall time: 3min 29s


In [64]:
def generate_text(model, start_string, num_words):
    model.eval()
    words = start_string.split()
    for _ in range(num_words):
        input_seq = (
            torch.LongTensor([words_to_idx[word] for word in words[-SEQUENCE_LENGTH:]])
            .unsqueeze(0)
            .to(device)
        )
        h, c = model.init_hidden(1)
        output, (h, c) = model(input_seq, (h, c))
        next_token = output.argmax(1)[-1].item()
        words.append(idx_to_word[next_token])
    return " ".join(words)


# Example usage:
print(
    "Generated Text:", generate_text(lstm_word_model, start_string="Alice was a", num_words=100)
)

Generated Text: Alice was a nice little histories and eyes bottle up you know.' 'Not you am, sir,' said the Mock Turtle went on a little pattering of feet on the King. 'I shall be free said she was not quite not to be sure, said the Mock Turtle went on a little pattering of feet on the King. 'I shall be free said she was not quite not to be sure, said the Mock Turtle went on a little pattering of feet on the King. 'I shall be free said she was not quite not to be sure, said the Mock Turtle went on


## Better Model word level LSTM
https://miro.medium.com/v2/resize:fit:828/format:webp/1*GQTsstXeg3-RIoExJ7JNzA.png

https://miro.medium.com/v2/resize:fit:828/format:webp/1*jR41vj92z59LBq4pATaE-Q.png

In [94]:
# Importing the dataset read in text format
def read_file_yield(filename):
    """Read the file form a text and split by para endings"""
    with open(filename, "r") as file:
        data = file.read()
        paragraphs = data.split('"\n')
        for para in paragraphs:
            # Remove leading and trailing whitespace and quotes
            yield para.strip('" \n')

In [97]:
# Example usage
train_path = "./data/wiki_short/train.txt"
test_path = "./data/wiki_short/test.txt"

def build_datasets(filename):
    datasets = []
    for paragraph in read_file_yield(filename):
        datasets.append(paragraph)

    return datasets

In [98]:
train_dataset = build_datasets(train_path)
test_dataset = build_datasets(test_path)

In [99]:
print(len(train_dataset))
print(len(test_dataset))

616
48


In [100]:
# tokenize the dataset
tokenizer = get_tokenizer(
    "basic_english"
)  ## We'll use tokenizer available from PyTorch


def build_vocab(datasets):
    for dataset in datasets:
        for text in dataset:
            yield tokenizer(text)

In [ ]:
vocab = build_vocab_from_iterator(build_vocab([train_dataset, test_dataset]), specials=["<UNK>",""])
vocab.set_default_index(vocab["<UNK>"])